In [ ]:
import torch
print(torch.cuda.is_available())
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Usando GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Usando CPU")

In [ ]:
from datasets import load_dataset
import os
import numpy as np

data_dir = 'data/jigsaw_toxicity_comment'
if not os.path.exists(f'{data_dir}/toxic_train_processed.csv'):
    # Cargar el dataset de Toxic Comment Classification Challenge
    import pandas as pd
    data_dir = 'data/jigsaw_toxicity_comment'
    # Cargar los archivos CSV
    train_df = pd.read_csv(f'{data_dir}/toxic_train.csv')

    # Dividir el dataset de entrenamiento en entrenamiento y validación
    from sklearn.model_selection import train_test_split

    train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

    # Guardar los archivos procesados
    train_df.to_csv(f'{data_dir}/toxic_train_processed.csv', index=False)
    val_df.to_csv(f'{data_dir}/toxic_val_processed.csv', index=False)


    # Cargar el dataset desde el directorio local
dataset = load_dataset('csv', data_files={'train': f'{data_dir}/toxic_train_processed.csv', 'validation': f'{data_dir}/toxic_val_processed.csv'})

In [ ]:
print(dataset)

## Usamos el tokenizador de Distibert para nuestra muestra

In [ ]:
from transformers import DistilBertTokenizerFast
import transformers
import accelerate
print("Transformers version:", transformers.__version__)
print("Accelerate version:", accelerate.__version__)
# Inicializamosel tokenizer de distilBERT
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# definimos las etiquetas
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Funcion de tokenización y perparacion de etiquetas
def tokenize_and_encode(examples):
    encodings = tokenizer(examples['comment_text'], padding='max_length', truncation=True)
    labels_matrix = np.zeros((len(examples['comment_text']), len(labels)))
    for i, label in enumerate(labels):
        labels_matrix[:, i] = examples[label]
    encodings['labels'] = labels_matrix.tolist()
    return encodings

# Aplicamos la tokenización y etiquetas procesada
tokenized_datasets = dataset.map(tokenize_and_encode, batched=True, remove_columns=['comment_text'] + labels)
#tokenized_datasets.set_format("torch")
train_dataset = tokenized_datasets['train']
val_dataset = tokenized_datasets['validation']



## Cargamos el modelo, la clase para entrenar y la clase de configuración
### En este caso lo preparamos para multietiqueta

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# Cargar el modelo preentrenado con el número adecuado de etiquetas
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(labels))


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = (pred.predictions > 0.5).astype(int)  # Convertir predicciones a binario usando un umbral

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# Si queremos continuar desde un chepoint
model = DistilBertForSequenceClassification.from_pretrained(f"{data_dir}/distilbert-finetuned-toxic")

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none"
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

## Entrenamos

In [ ]:

trainer.train(resume_from_checkpoint=True)

In [ ]:
results = trainer.evaluate()
print(results)

In [ ]:
model.save_pretrained(f"{data_dir}/distilbert-finetuned-toxic")
tokenizer.save_pretrained(f"{data_dir}/distilbert-finetuned-toxic")

In [ ]:
import pandas as pd
df_labels_test = pd.read_csv(f"{data_dir}/test_labels.csv")
df_datos_test = pd.read_csv(f"{data_dir}/toxic_test.csv")
df_conjunto = df_datos_test.merge(df_labels_test, how='left', on='id')
df_conjunto_filtrado = df_conjunto[~(df_conjunto[labels] == -1).all(axis=1)]
df_conjunto_filtrado = df_conjunto_filtrado.drop(['id'], axis=1)
df_conjunto_filtrado = df_conjunto_filtrado.reset_index(drop=True)
df_conjunto_filtrado.head()

In [ ]:

import random

# Verificando si hay una GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

correctos = 0
aceptables = 0
incorrectos = 0
# Movemos el modelo al dispositivo
model = model.to(device)
for _ in range(1000):
    random_index = random.randint(0, df_conjunto_filtrado.shape[0] - 1)

    fila = df_conjunto_filtrado.iloc[random_index]
    # Texto de ejemplo para probar el modelo
    text = fila['comment_text']

    # Tokenizar el texto
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    #movemos los tensores de entrada al dispositivo
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Deshabilitar el cálculo de gradientes para ahorrar memoria y mejorar la velocidad
    with torch.no_grad():
        outputs = model(**inputs)

    # kas salidas contienen los logits
    logits = outputs.logits

    # obtenemos las probabilidades con sigmoid a partir de los logits
    probabilities = torch.sigmoid(logits).cpu().numpy()[0]

    # sacamos las etiquetas predichas (usando un umbral, por ejemplo 0.5)
    predicted_labels = probabilities > 0.5

    # asociamso etiquetas predichas con etiquetas reales
    prediccion_valores = [labels[i] for i in range(len(labels)) if predicted_labels[i]]
    valores_esperados = [labels[i] for i in range(len(labels)) if fila[labels[i]] == 1]
    set_prediccion = set(prediccion_valores)
    set_esperados = set(valores_esperados)

    if set_esperados == set_prediccion:
        print(f"✅ Correcto {valores_esperados}")
        correctos += 1
    elif set_esperados.issubset(set_prediccion) and set_esperados and not set_prediccion.issubset(set_esperados):
        aceptables += 1
        print(f"😊 Esperado {valores_esperados} -- Prediccion {prediccion_valores}")
    else:
        incorrectos += 1
        print(f"❌ Esperado {valores_esperados} -- Prediccion {prediccion_valores}", f"Texto: {fila['comment_text']}")
    #print(f"Probabilities: {probabilities}")
    #print(f"Predicted labels: {predicted_labels}")
    #print(f"Predicted label names: {predicted_labels_names}")


In [ ]:
print("Correctos", correctos)
print("Aceptables", aceptables)
print("Incorrectos", incorrectos)
print("Relevancia Aprox", round((correctos + aceptables) / (correctos + aceptables + incorrectos) * 100, 2), "%")
print("Aciertos 100% Aprox", round((correctos) / (correctos + aceptables + incorrectos) * 100, 2), "%")